In [1]:
import re
import json

M3U_FILE = "tung_iptv.m3u"
OUTPUT_JSON = "tung_iptv.json"

FALLBACK_IMAGE = "https://raw.githubusercontent.com/lamtung16/iptv/refs/heads/main/images/logo.png"


def slugify(text):
    return re.sub(r'[^a-z0-9]+', '-', text.lower()).strip('-')


def stream_type(url):
    return "hls" if ".m3u8" in url else "direct"


def image_obj(url=None, img_type="contain", w=64, h=64):
    return {
        "url": url or FALLBACK_IMAGE,
        "type": img_type,
        "width": w,
        "height": h
    }


def parse_m3u(path):
    groups = {}
    current = None

    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()

            if not line or line.startswith("#EXTM3U"):
                continue

            if line.startswith("#EXTINF"):
                group = re.search(r'group-title="([^"]+)"', line)
                logo = re.search(r'tvg-logo="([^"]+)"', line)
                name = line.split(",", 1)[-1].strip()

                current = {
                    "group": group.group(1) if group else "Others",
                    "name": name,
                    "logo": logo.group(1) if logo else None
                }

            elif current and not line.startswith("#"):
                group_name = current["group"]
                groups.setdefault(group_name, [])

                cid = slugify(current["name"])

                groups[group_name].append({
                    "id": cid,
                    "name": current["name"],
                    "type": "single",
                    "display": "default",
                    "image": image_obj(current["logo"]),
                    "enable_detail": False,
                    "labels": [],
                    "share": None,
                    "remote_data": None,
                    "sources": [
                        {
                            "id": "main",
                            "name": "Main",
                            "image": image_obj(),
                            "contents": [
                                {
                                    "id": "live",
                                    "name": "Live",
                                    "image": image_obj(),
                                    "grid_number": 1,
                                    "streams": [
                                        {
                                            "id": f"{cid}-stream",
                                            "name": "Live",
                                            "image": image_obj(),
                                            "stream_links": [
                                                {
                                                    "id": f"{cid}-link",
                                                    "name": "Server 1",
                                                    "url": line,
                                                    "type": stream_type(line),
                                                    "default": True,
                                                    "enableP2P": False,
                                                    "subtitles": None,
                                                    "remote_data": None,
                                                    "request_headers": None,
                                                    "comments": None
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ],
                            "remote_data": None
                        }
                    ]
                })

                current = None

    return groups


def build_json(groups):
    return {
        "id": "tung-iptv",
        "name": "Tung IPTV",
        "description": "Sports",
        "url": None,
        "color": "#1E90FF",
        "disable_ads": True,
        "grid_number": 2,
        "image": {
            "type": "cover",
            "url": FALLBACK_IMAGE
        },
        "groups": [
            {
                "id": slugify(name),
                "name": name,
                "image": image_obj(FALLBACK_IMAGE, "cover"),
                "channels": channels
            }
            for name, channels in groups.items()
        ]
    }


if __name__ == "__main__":
    groups = parse_m3u(M3U_FILE)
    data = build_json(groups)

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"✅ SUCCESS: {len(groups)} groups written to {OUTPUT_JSON}")

✅ SUCCESS: 4 groups written to tung_iptv.json
